# EVLUATION OF POLETTI SIMULATIONS

This program is used to evaluate the simulations for the experiments conducted by Poletti et al. It compares spiking events in the center with spikes in the outer region. If both regions show spikes during time frame of duration $t_{bin}$, there is no differential motion counted.

In [ ]:
import pylab as pyl
import numpy as np
import cv2
import sys
import os

if len(sys.argv)==9:
    sim_title = sys.argv[1]
    sim_title_2 = sys.argv[2]
    sim_nr = sys.argv[3]
    handle_name = sys.argv[4]
    extent_x = float(sys.argv[5])-1.
    extent_y = float(sys.argv[6])-1.
    exp_nr = sys.argv[7]
    cond_nr = sys.argv[8]
    sar=True
else:
    sim_title = sys.argv[1]
    sim_nr = sys.argv[2]
    handle_name = sys.argv[3]
    extent_x = float(sys.argv[4])
    extent_y = float(sys.argv[5])-1.
    exp_nr = int(float(sys.argv[6]))
    cond_nr = int(float(sys.argv[7]))
    sim_title_2 = ''
    sar=False

Setting constants

In [ ]:
#for 5ms binning in order to reduce noise 
bin = 15

t_len = 600.
#border zone attributed to the frame
border = 0.3
#threshold for number of spikes in interval counting as coincidental
thr = 0
border_thr = 0
#for exp2 cond 7/8
if cond_nr == (7 or 8):
	vel = 0.06
else:
	vel = 0.

width = extent_x

There are two possible evaluation mechanisms: Checking motion detector responses or parasol cells. 

### Motion detector dependent

In [ ]:
#load the spike data
fl = open('/data/'+str(sim_title)+'network/'+str(sim_nr)+'/ml_times_pos.txt','r+')
fr = open('/data/'+str(sim_title)+'network/'+str(sim_nr)+'/mr_times_pos.txt','r+')

#files to write to
rr = open('/data/'+str(sim_title)+'network/'+str(sim_nr)+'/coincidence_right_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','w+')
rl = open('/data/'+str(sim_title)+'network/'+str(sim_nr)+'/coincidence_left_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','w+')
rc_all = open('/data/'+str(sim_title)+'network/'+str(sim_nr)+'/coincidence_all_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','w+')
rc_count_all = open('/data/poletti2010/poletti2010_coincidence_count_all_t90_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','a+')

#leftward motion - split inner and outer region
time_inner_arr=np.zeros(int(t_len/bin))
time_outer_arr=np.zeros(int(t_len/bin))

#bins filled with 1 if differential motion, otherwise 0
#leftward
coincidence_arr=np.zeros(int(t_len/bin))
#rightward
coincidence_r_arr=np.zeros(int(t_len/bin))

Now handle the files for leftward and rightward motion

In [ ]:
#leftward motion
for line in fr:
    #handle input file
	if len(line.split(','))>0:
		line = line.split(')')[0]
		line = line.split('(')[1]
        #get time and position
		t = int(line.split(',')[0])
		xpos = float(line.split(',')[2])
        #omit first bins in order to leave out starting dynamics
		if t > 90:
            #split between inner and outer region
			if xpos > (border*width + vel*float(t)) and xpos < ((1.-border)*width + vel*float(t)):
				time_inner_arr[int(float(t)/bin)] += 1
			else:
				time_outer_arr[int(float(t)/bin)] += 1

#if the threshold of the number of spikes within a bin is larger than a threshold thr, add to coincidence list
for t in range(int(t_len/bin)):
	if time_inner_arr[t] > thr and time_outer_arr[t] <= border_thr:
		coincidence_r_arr[t] = 1
	if time_inner_arr[t] <= thr and time_outer_arr[t] > border_thr:
		coincidence_r_arr[t] = 1
	rr.write(str(t*bin)+'\t'+str(time_inner_arr[t])+'\t'+str(time_outer_arr[t])+'\t'+str(coincidence_r_arr[t])+'\n')
	
    
    
#rightward motion -> as for leftward
time_inner_arr=np.zeros(int(t_len/bin))
time_outer_arr=np.zeros(int(t_len/bin))
coincidence_l_arr=np.zeros(int(t_len/bin))
for line in fl:
	if len(line.split(','))>0:
		line = line.split(')')[0]
		line = line.split('(')[1]
		t = int(line.split(',')[0])
		xpos = float(line.split(',')[2])
		if t > 90:
			if xpos > (border*width + vel*float(t)) and xpos < ((1.-border)*width + vel*float(t)):
				time_inner_arr[int(float(t)/bin)] += 1
			else:
				time_outer_arr[int(float(t)/bin)] += 1

for t in range(int(t_len/bin)):
	if time_inner_arr[t] > thr and time_outer_arr[t] <= border_thr:
		coincidence_l_arr[t] = 1
		coincidence_arr[t] = 1
	if time_inner_arr[t] <= thr and time_outer_arr[t] > border_thr:
		coincidence_l_arr[t] = 1
		coincidence_arr[t] = 1
	rl.write(str(t*bin)+'\t'+str(time_inner_arr[t])+'\t'+str(time_outer_arr[t])+'\t'+str(coincidence_l_arr[t])+'\n')
	
#coincidence_arr = coincidence_l_arr*coincidence_r_arr

#count number of intervals with 1 
ca_count = 0
for t in range(int(t_len/bin)):
	if coincidence_arr[t] == 1:
		ca_count += 1
	rc_all.write(str(t*bin)+'\t'+str(coincidence_arr[t])+'\n')	
	
rc_count_all.write(str(exp_nr)+'\t'+str(cond_nr)+'\t'+str(sim_nr)+'\t'+str(ca_count)+'\n')	

rr.close()
rl.close()
rc_all.close()
rc_count_all.close()

### Parasol dependent

In [ ]:
#FOR PARASOLS INSTEAD
fp = open('/home/schrader/Documents/microsaccades/data/'+str(sim_title)+'network/'+str(sim_nr)+'/par_times_pos.txt','r+')
rc_count_all = open('/home/schrader/Documents/microsaccades/data/poletti2010/poletti2010_coincidence_parasol_t90_count_all_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','a+')

rp = open('/home/schrader/Documents/microsaccades/data/'+str(sim_title)+'network/'+str(sim_nr)+'/coincidence_parasol_bin'+str(bin)+'_border'+str(int(100*border))+'_thr'+str(thr)+'_bthr'+str(border_thr)+'.txt','w+')

#leftward motion
time_inner_arr=np.zeros(int(t_len/float(bin)))
time_outer_arr=np.zeros(int(t_len/float(bin)))

coincidence_arr=np.zeros(int(t_len/float(bin)))


for line in fp:
	if len(line.split(','))>0:
		line = line.split(')')[0]
		line = line.split('(')[1]
		t = int(line.split(',')[0])
		xpos = float(line.split(',')[2])
		if t > 90:
			if xpos > (border*width + vel*float(t)) and xpos < ((1.-border)*width + vel*float(t)):
				time_inner_arr[int(float(t)/float(bin))] += 1
			else:
				time_outer_arr[int(float(t)/float(bin))] += 1

for t in range(int(t_len/float(bin))):
	if time_inner_arr[t] > thr and time_outer_arr[t] <= border_thr:
		coincidence_arr[t] = 1
	if time_inner_arr[t] <= thr and time_outer_arr[t] > border_thr:
		coincidence_arr[t] = 1
	rp.write(str(t*bin)+'\t'+str(time_inner_arr[t])+'\t'+str(time_outer_arr[t])+'\t'+str(coincidence_arr[t])+'\n')

ca_count = 0
for t in range(int(t_len/float(bin))):
	if coincidence_arr[t] == 1:
		ca_count += 1
	
rc_count_all.write(str(exp_nr)+'\t'+str(cond_nr)+'\t'+str(sim_nr)+'\t'+str(ca_count)+'\n')	

rp.close()
rc_count_all.close()